In [1]:
import pandas as pd
import numpy as np
import glob
import os
import pickle
from decouple import Config, RepositoryEnv
from sqlalchemy import create_engine



def _clean_results_data(combined_results_df) -> pd.DataFrame:
    """
    Function to clean results data.
    """
    
    #combined_results_unnamed_removed_df = _remove_unnamed_values(combined_results_df)
    #combined_results_remove_invalid_scores_df = _remove_invalid_scores(combined_results_unnamed_removed_df)
    #combined_results_standarised_match_links_df = _standardise_match_links(combined_results_remove_invalid_scores_df)
    combined_results_data_types_df = _convert_results_data_types(combined_results_df)

    return combined_results_data_types_df


def _convert_results_data_types(combined_results_df):
    """
    Function to update data types of columns to the erquired formats.
    """
    # Update data types
    combined_results_df["Home_Team"] = combined_results_df["Home_Team"].astype('category')
    combined_results_df["Away_Team"] = combined_results_df["Away_Team"].astype('category')
    combined_results_df["Home_Goals"] = combined_results_df["Home_Goals"].astype('int')
    combined_results_df["Away_Goals"] = combined_results_df["Away_Goals"].astype('int')
    combined_results_df["Season"] = combined_results_df["Season"].astype('int')
    combined_results_df["Round"] = combined_results_df["Round"].astype('int')
    combined_results_df["League"] = combined_results_df["League"].astype('string')
    combined_results_df["ELO_Home"] = combined_results_df["ELO_Home"].astype('int')
    combined_results_df["ELO_Away"] = combined_results_df["ELO_Away"].astype('int')

    #combined_results_df["Link"] = combined_results_df["Link"].astype('string')

    return combined_results_df

def _remove_unnamed_values(combined_results_df):
    """
    Function to remove presence of unknown variable
    """
    # remove unnamed values
    combined_results_df.drop("Unnamed: 0", axis = 1, inplace=True)
    return combined_results_df

def _remove_invalid_scores(combined_results_df):
    '''_remove_invalid_scores 
    Function to remove invalid scores.

    A number of scores are in invlid formats. This function modifies wha is known and removes what isn't.

    Arguments:
        combined_results_df -- Combined results dataframe

    Returns:
        updated combined results dataframe
    '''

    # identify and remove invalid scores

    invalid_score_1 = (combined_results_df.loc[: , "Result"].str.contains(":"))
    invalid_score_2 = (combined_results_df.loc[: , "Result"].str.len() > 5)
    invalid_score_3 = (combined_results_df.loc[: , "Result"].str.len() < 2)

    combined_results_df.replace("1 (0-0) 1", np.NaN, inplace=True)
    combined_results_df.replace("0 (0-0) 0", "0-0", inplace=True)
    combined_results_df.replace("3 (3-2) 2", "3-2", inplace=True)
    combined_results_df.replace("0 (0-1) 1", "0-1", inplace=True)

    combined_results_df[invalid_score_1] = np.NaN
    combined_results_df[invalid_score_2] = np.NaN
    combined_results_df[invalid_score_3] = np.NaN

    combined_results_df.dropna(subset=["Result"], inplace = True)
    
    return combined_results_df

def _standardise_match_links(combined_results_df):
    '''_standardise_match_links 
    Function to standardise the links to enable merging between datasets.
    '''
    combined_results_df["Link"] = combined_results_df["Link"].apply(lambda x: "".join(x.replace(x.split("/")[6], "")) + x.split("/")[6][0:4])
    
    return combined_results_df

def _remove_partial_complete_league(combined_ELO_df):
    '''_remove_partial_complete_league 
    Function to remove presence of leagues that contain only one round of results

    Arguments:
        combined_results_df -- _description_
    '''
    for league in combined_ELO_df["League"].unique():
        for season in combined_ELO_df["Season"].unique():
            if len(combined_ELO_df.loc[(combined_ELO_df.League == league)& (
                combined_ELO_df.Season == season)]["Round"].unique()) <= 1:
                combined_ELO_df.drop(combined_ELO_df.index[(combined_ELO_df.League == league)&(
                    combined_ELO_df.Season == season)], inplace=True)
    
    return combined_ELO_df


def _clean_match_info_data(match_info_df) -> pd.DataFrame:
    '''clean_match_info_data 
    

    Returns:
        _description_
    '''
    match_info_card_df = _replace_missing_card_data(match_info_df)
    match_info_data_types_converted_df = _convert_match_info_data_types(match_info_card_df)
    match_info_standardised_df = _standardise_referee_and_links(match_info_data_types_converted_df)

    return match_info_standardised_df

def _replace_missing_card_data(match_info_df):
    '''_replace_missing_card_data _summary_

    Arguments:
        match_info_df -- _description_
    '''
    match_info_df["Home_Yellow"].replace(np.NaN, 0, inplace=True)
    match_info_df["Home_Red"].replace(np.NaN, 0, inplace=True)
    match_info_df["Away_Yellow"].replace(np.NaN, 0, inplace=True)
    match_info_df["Away_Red"].replace(np.NaN, 0, inplace=True)

    return match_info_df

def _convert_match_info_data_types(match_info_df):
    '''_convert_data_types 

    Arguments:
        match_info_df -- _description_
    '''
    match_info_df["Date_New"] = match_info_df["Date_New"].astype('datetime64')
    match_info_df["Referee"] = match_info_df["Referee"].astype('string')
    match_info_df["Home_Yellow"] = match_info_df["Home_Yellow"].astype('int')
    match_info_df["Home_Red"] = match_info_df["Home_Red"].astype('int')
    match_info_df["Away_Yellow"] = match_info_df["Away_Yellow"].astype('int')
    match_info_df["Away_Red"] = match_info_df["Away_Red"].astype('int')
    match_info_df["Link"] = match_info_df["Link"].astype('string')

    return match_info_df

def _standardise_referee_and_links(match_info_df):
    '''_standardise_referee_and_links _summary_

    Arguments:
        match_info_df -- _description_
    '''
    match_info_df["Referee"] = match_info_df["Referee"].apply(lambda x: (x.replace("\r\n", "")).replace("Referee: ", ""))
    match_info_df["Link"] = match_info_df["Link"].apply(lambda x: "https://www.besoccer.com" + x)

    return match_info_df

def _clean_team_info_data(team_info_df):
    '''clean_team_info_data 

    Arguments:
        team_info_df -- _description_
    '''
    team_info_df["Capacity"] = team_info_df["Capacity"].apply(lambda x: x.replace(",", ""))
    team_info_df["Capacity"] = team_info_df["Capacity"].astype('int')

    # rename column in team info

    team_info_df = team_info_df.rename(columns = {'Team' : 'Home_Team'})

    return team_info_df

def clean_all_dataframes(combined_results_df):
    combined_results_cleaned_df = _clean_results_data(combined_results_df)
    #match_info_cleaned_df = _clean_match_info_data(match_info_df)
    #team_info_cleaned_df = _clean_team_info_data(team_info_df)

    return combined_results_cleaned_df

def add_features_to_results_df(combined_results_cleaned_df):
    '''add_features_to_results_df _summary_

    Arguments:
        combined_results_cleaned_df -- _description_
    '''

    combined_results_cleaned_df["Home_Win"] = combined_results_cleaned_df["Home_Goals"] > combined_results_cleaned_df["Away_Goals"]
    combined_results_cleaned_df["Away_Win"] = combined_results_cleaned_df["Home_Goals"] < combined_results_cleaned_df["Away_Goals"]


    return combined_results_cleaned_df

def combine_dataframes(combined_results_cleaned_df, match_info_cleaned_df, team_info_cleaned_df) -> pd.DataFrame:
    '''combine_dataframes _summary_

    Arguments:
        combined_results_cleaned_df -- _description_
        match_info_cleaned_df -- _description_
        team_info_cleaned_df -- _description_

    Returns:
        _description_
    '''
    #result_match_df = pd.merge(combined_results_cleaned_df, match_info_cleaned_df, how='inner', on="Link")
    combined_df = pd.merge(combined_results_cleaned_df, team_info_cleaned_df, how='outer', on="Home_Team")

    return combined_df

def _load_ELO_pickle_data() -> pd.DataFrame:
    ELO_dict = pickle.load(open('2. Feature Engineering/elo_dict.pkl', 'rb'))
    ELO_df = pd.DataFrame(ELO_dict.items(), columns=["Link", "ELO_dict"])

    return ELO_df

def _clean_EDA_ELO_data(ELO_df):
    ELO_df["ELO_home"] = (ELO_df["ELO_dict"]).apply(lambda x: x['Elo_home'])
    ELO_df["ELO_away"] = (ELO_df["ELO_dict"]).apply(lambda x: x['Elo_away'])
    ELO_df.drop(columns="ELO_dict", inplace=True)
    ELO_df["Link"] = ELO_df["Link"].apply(lambda x: "".join(x.replace(x.split("/")[6], "")) + x.split("/")[6][0:4])

    return ELO_df

def _merge_ELO_combined_df(combined_df, ELO_df):

    combined_ELO_df = pd.merge(combined_df, ELO_df, how='inner', on="Link")

    return combined_ELO_df

def load_clean_merge_ELO_data(combined_df):

    ELO_df = _load_ELO_pickle_data()
    ELO_cleaned_df = _clean_EDA_ELO_data(ELO_df)
    combined_ELO_df = _merge_ELO_combined_df(combined_df, ELO_cleaned_df)

    return combined_ELO_df


def create_new_feature_columns(combined_elo_df):
    '''create_new_feature_columns 
    Function to create columns filled with zeros for each required feature

    Arguments:
        combined_local_elo_df -- _description_
    '''
    combined_elo_df["home_team_total_goals_scored_so_far"] = 0
    combined_elo_df["home_team_total_goals_conceeded_so_far"] = 0
    combined_elo_df["home_team_current_win_streak"] = 0
    combined_elo_df["home_team_current_loss_streak"] = 0
    combined_elo_df["home_team_total_points_so_far"] = 0
    combined_elo_df["home_team_current_goal_drought"] = 0
    combined_elo_df["home_team_total_wins_so_far"] = 0

    combined_elo_df["away_team_total_goals_scored_so_far"] = 0
    combined_elo_df["away_team_total_goals_conceeded_so_far"] = 0
    combined_elo_df["away_team_current_win_streak"] = 0
    combined_elo_df["away_team_current_loss_streak"] = 0
    combined_elo_df["away_team_total_points_so_far"] = 0
    combined_elo_df["away_team_current_goal_drought"] = 0
    combined_elo_df["away_team_total_wins_so_far"] = 0

    return combined_elo_df

def group_by_league_df(combined_elo_df: pd.DataFrame, league) -> pd.DataFrame:
    """
    Function to group full dataframe into only the required league

    """
    grouped_df = combined_elo_df.groupby("League")
    league_df = grouped_df.get_group(league)

    return league_df


def group_by_season_df(league_df: pd.DataFrame, season) -> pd.DataFrame:
    """
    Function to group full dataframe into seasons

    """

    grouped_df = league_df.groupby("Season")
    season_df = grouped_df.get_group(season)
    # Order season by round
    season_df = season_df.sort_values(by=['Round'], ascending=[True])

    return season_df


def create_summary_template(season_df):
    '''create_summary_template 
    Function to create auxiliary template summary per season

    Arguments:
        season_df -- _description_

    Returns:
        _description_
    '''

    summary_df_template = pd.DataFrame(columns=
    [
        'team_name', 'season', 'league', 'total_goals_scored_so_far', 'total_goals_conceeded_so_far', 
        'recent_win_streak', 'recent_loss_streak', 'total_points_so_far', 'recent_goal_drought', 
        'total_wins_so_far'
        ])
    team_group = season_df.groupby("Home_Team")

    for home_team_name in season_df["Home_Team"].unique():
        team_df = team_group.get_group(home_team_name)     
        team_name = home_team_name
        season = team_df["Season"].unique()[0]
        league = team_df["League"].unique()[0]

        team_stats_dict = {
            'team_name': team_name, 'season': season, 
            'league': league, 'total_goals_scored_so_far': 0, 
            'total_goals_conceeded_so_far': 0, 'recent_win_streak': 0,
            'recent_loss_streak': 0, 'total_points_so_far': 0, 
            'recent_goal_drought': 0, 'total_wins_so_far': 0
            }
        team_stats_dict_df = pd.DataFrame([team_stats_dict])
        summary_df_template = pd.concat([summary_df_template, team_stats_dict_df], ignore_index=True)


    print("Summary Template Complete")
    return summary_df_template

def populate_season_stats(season_df, summary_df_template):
    '''populate_season_stats 
    Function to add to season templte summary per round.

    Arguments:
        season_df -- _description_
        summary_df_template -- _description_

    Returns:
        _description_
    '''
    round_df_grouped = season_df.groupby("Round")

    for round in season_df["Round"].unique():
        round_df = round_df_grouped.get_group(round)
        # Set the values of the previous match
        for team in list(summary_df_template["team_name"]):

            populated_season_df = _add_home_team_feature_total_to_df("home_team_total_goals_scored_so_far","total_goals_scored_so_far", team, season_df, summary_df_template, round)
            populated_season_df = _add_home_team_feature_total_to_df("home_team_total_goals_conceeded_so_far","total_goals_conceeded_so_far", team, season_df, summary_df_template, round)
            populated_season_df = _add_home_team_feature_total_to_df("home_team_total_points_so_far","total_points_so_far", team, season_df, summary_df_template, round)
            populated_season_df = _add_home_team_feature_total_to_df("home_team_total_wins_so_far","total_wins_so_far", team, season_df, summary_df_template, round)

            populated_season_df = _add_away_team_feature_total_to_df("away_team_total_goals_scored_so_far","total_goals_scored_so_far", team, season_df, summary_df_template, round)
            populated_season_df = _add_away_team_feature_total_to_df("away_team_total_goals_conceeded_so_far","total_goals_conceeded_so_far", team, season_df, summary_df_template, round)
            populated_season_df = _add_away_team_feature_total_to_df("away_team_total_points_so_far","total_points_so_far", team, season_df, summary_df_template, round)
            populated_season_df = _add_away_team_feature_total_to_df("away_team_total_wins_so_far","total_wins_so_far", team, season_df, summary_df_template, round)

            
    
        for index, row in round_df.iterrows():

            home_team_name = row["Home_Team"]
            away_team_name = row["Away_Team"]
            season = row["Season"]
            league = row["League"]
            home_goals_scored = row["Home_Goals"]
            away_goals_scored = row["Away_Goals"]
        

            # Home Team
            summary_df_template = _goals_and_points_totaller(summary_df_template, home_team_name, home_goals_scored, away_goals_scored, season, league)
            # Away Team
            summary_df_template = _goals_and_points_totaller(summary_df_template, away_team_name, away_goals_scored, home_goals_scored, season, league)

        print(f"Summary DF Template Populated for Round {round}")
    
    print(f"Summary DF Template Populated for season {season}")


    return summary_df_template, populated_season_df

def _goals_and_points_totaller(summary_df_template, team, goal_option_1, goal_option_2, season, league):

    summary_df_template.loc[((summary_df_template.team_name == team) & (
    summary_df_template.season == season) & (
    summary_df_template.league == league)), "total_goals_scored_so_far"] += goal_option_1

    summary_df_template.loc[((summary_df_template.team_name == team) & (
        summary_df_template.season == season) & (
        summary_df_template.league == league)), "total_goals_conceeded_so_far"] += goal_option_2
        
    if goal_option_1 > goal_option_2:

        summary_df_template.loc[((summary_df_template.team_name == team) & (
        summary_df_template.season == season) & (
        summary_df_template.league == league)), "total_points_so_far"] += 3
        summary_df_template.loc[((summary_df_template.team_name == team) & (
        summary_df_template.season == season) & (
        summary_df_template.league == league)), "total_wins_so_far"] += 1
    
    if goal_option_1 == goal_option_2:

        summary_df_template.loc[((summary_df_template.team_name == team) & (
        summary_df_template.season == season) & (
        summary_df_template.league == league)), "total_points_so_far"] += 1
    
    return summary_df_template

def _add_home_team_feature_total_to_df(feature,summary_feature, team, df, summary_df_template, round):
    
    df.loc[(df.Home_Team == team) & ((
                df.Round == round)), feature] = summary_df_template.loc[(
                    summary_df_template.team_name == team), summary_feature].values[0]
    return df

def _add_away_team_feature_total_to_df(feature,summary_feature, team, df, summary_df_template, round):
    
    df.loc[((df.Away_Team == team) & (
                df.Round == round)), feature] = summary_df_template.loc[((
                    summary_df_template.team_name == team)), summary_feature].values[0]
    return df


def populate_streak_season_stats(summary_df_populated, populated_season_df, season, league, missing_data_information_list):
    '''populate_streak_season_stats 
    Function to calculate the recent form going into game, such as win / loss streaks as well as goal drought.

    Arguments:
        summary_df_populated -- _description_
        populated_season_df -- _description_

    Returns:
        _description_
    '''

    home_team_group = populated_season_df.groupby("Home_Team")
    away_team_group = populated_season_df.groupby("Away_Team")

    for team in list(summary_df_populated["team_name"]):

        try:
        
            home_team_df = home_team_group.get_group(team)
            away_team_df = away_team_group.get_group(team)
            team_df = pd.concat([home_team_df, away_team_df], axis=0, ignore_index=True)
            team_df = team_df.sort_values(by=['Round'], ascending=[True])
            win_streak = 0
            loss_streak = 0
            drought_streak = 0

            round_df_grouped = team_df.groupby("Round")
            print(f"Completing Streak Calculation for {team}...")
            for round in team_df["Round"].unique():
                round_df = round_df_grouped.get_group(round)
                # Here so that is shows for the round after 

                populated_season_df = _add_home_team_feature_total_to_df("home_team_current_win_streak","recent_win_streak", team, populated_season_df, summary_df_populated, round)
                populated_season_df = _add_home_team_feature_total_to_df("home_team_current_loss_streak","recent_loss_streak", team, populated_season_df, summary_df_populated, round)
                populated_season_df = _add_home_team_feature_total_to_df("home_team_current_goal_drought","recent_goal_drought", team, populated_season_df, summary_df_populated, round)
            
                populated_season_df = _add_away_team_feature_total_to_df("away_team_current_win_streak","recent_win_streak", team, populated_season_df, summary_df_populated, round)
                populated_season_df = _add_away_team_feature_total_to_df("away_team_current_loss_streak","recent_loss_streak", team, populated_season_df, summary_df_populated, round)
                populated_season_df = _add_away_team_feature_total_to_df("away_team_current_goal_drought","recent_goal_drought", team, populated_season_df, summary_df_populated, round)

                    
                for index, row in round_df.iterrows():
                    
                    home_team_name = row["Home_Team"]
                    away_team_name = row["Away_Team"]
                    season = row["Season"]
                    league = row["League"]
                    home_win = row["Home_Win"]
                    away_win = row["Away_Win"]
                    home_goals_scored = row["Home_Goals"]
                    away_goals_scored = row["Away_Goals"]

                    # Winning Streak

                    if (home_team_name == team and home_win == True) or (away_team_name == team and away_win == True):
                        win_streak += 1
                    
                    if (home_team_name == team and home_win == False) or (away_team_name == team and away_win == False):
                        win_streak = 0
                
                    # Losing Streak     
                    if (home_team_name == team and home_goals_scored < away_goals_scored) or (away_team_name == team and home_goals_scored > away_goals_scored):
                        loss_streak += 1
                    if (home_team_name == team and home_goals_scored > away_goals_scored) or (away_team_name == team and home_goals_scored < away_goals_scored):
                        loss_streak = 0

                    # Goal Drought
                    if (home_team_name == team and home_goals_scored == 0) or (away_team_name == team and away_goals_scored == 0):
                        drought_streak += 1
                    if (home_team_name == team and home_goals_scored > 0) or (away_team_name == team and away_goals_scored > 0):
                        drought_streak = 0

                    summary_df_populated.loc[((summary_df_populated.team_name == team) & (
                        summary_df_populated.season == season) & (
                        summary_df_populated.league == league)), "recent_win_streak"] = win_streak
                    
                    summary_df_populated.loc[((summary_df_populated.team_name == team) & (
                        summary_df_populated.season == season) & (
                        summary_df_populated.league == league)), "recent_loss_streak"] = loss_streak

                    summary_df_populated.loc[((summary_df_populated.team_name == team) & (
                        summary_df_populated.season == season) & (
                        summary_df_populated.league == league)), "recent_goal_drought"] = drought_streak
        except KeyError:
            print(f"The league {league} in season {season} appears incomplete")
            missing_data_information_list.append(f"{league}: {season}")

            
    print(f"Streak Stats Completed for season {season}")
            

    return summary_df_populated, populated_season_df, missing_data_information_list

def merge_season_df_with_combined_df(combined_ELO_df, populated_season_df):
    combined_ELO_df.update(populated_season_df, overwrite=True)

    return combined_ELO_df

# Function to remove partially completed seasons

def _remove_incomplete_seasons(missing_information_list, combined_elo_with_features_df):

    missing_info_set = set(missing_information_list)
    missing_info_deduplicated_list = list(missing_info_set)
    missing_info_deduplicated_list
    for item in missing_info_deduplicated_list:
        league = item.split(": ")[0]
        season = item.split(": ")[1]
        combined_elo_with_features_df.drop(combined_elo_with_features_df.index[(combined_elo_with_features_df.League == league)&(
                        combined_elo_with_features_df.Season == season)], inplace=True)
    
    return combined_elo_with_features_df

def _prepare_dataset_for_ML_models(df):
    '''_prepare_dataset_for_ML_models 
    Function to prepare data for ML models. This includes:

    1. Removing columns containing non-numerical data
    2. Creating a goal difference column as the target
    3. Drop Home and Away Goals as that is the result.
    4. Drop fixtures that do not contain ELO scores

    Arguments:
        df -- _description_

    Returns:
        _description_
    '''


    df.drop(axis=1, columns=['Result','Round', 'Link', 'Home_Win', 'Away_Win', 'Date_New', 'Home_Team', 'Away_Team', 'Season', 'League', 'Referee', 'City', 'Country', 'Stadium', 'Pitch','Home_Yellow', 'Home_Red', 'Away_Yellow', 'Away_Red'], inplace=True)
    df["Goal_Difference"] = df["Home_Goals"] - df["Away_Goals"]
    df.drop(axis=1, columns=['Home_Goals', 'Away_Goals'], inplace=True)
    df.dropna(subset=["ELO_home"], inplace = True)
    
    return df

# Group into decades

def _split_dataset_into_decades(combined_elo_with_partial_seasons_removed_df):
    '''split_dataset_into_decades 
    Function to split dataset into three datasets each approximately 10 years long
    '''

    combined_elo_with_partial_seasons_removed_df = combined_elo_with_partial_seasons_removed_df.sort_values(by=['Season'], ascending=[True])

    decade_1 = (combined_elo_with_partial_seasons_removed_df["Season"].unique()[0:10])
    decade_2 = (combined_elo_with_partial_seasons_removed_df["Season"].unique()[10:20])
    decade_3 = (combined_elo_with_partial_seasons_removed_df["Season"].unique()[20:32])

    season_grouped_df = combined_elo_with_partial_seasons_removed_df.groupby("Season")

    decade_1_df = pd.DataFrame()
    decade_2_df = pd.DataFrame()
    decade_3_df = pd.DataFrame()

    for season in decade_1:
        decade_df = season_grouped_df.get_group(season)
        decade_1_df = pd.concat([decade_1_df, decade_df], axis=0, ignore_index=True)

    for season in decade_2:
        decade_df = season_grouped_df.get_group(season)
        decade_2_df = pd.concat([decade_2_df, decade_df], axis=0, ignore_index=True)

    for season in decade_3:
        decade_df = season_grouped_df.get_group(season)
        decade_3_df = pd.concat([decade_3_df, decade_df], axis=0, ignore_index=True)


    return decade_1_df, decade_2_df, decade_3_df

def _group_into_divisions(decade_df, division):
    '''_group_into_leagues _summary_

    Arguments:
        decade_df -- _description_
        division -- _description_
    '''

    league_grouped_df = decade_df.groupby("League")
    division_df = league_grouped_df.get_group(division)

    return division_df

def _create_average_feature_columns(split_df):
    '''create_new_feature_columns 
    Function to create columns filled with zeros for each required feature

    Arguments:
        combined_local_elo_df -- _description_
    '''
    # Get an average goals_scored and conceeded

    split_df["home_team_average_goals_scored_per_game"] = 0
    split_df["home_team_average_goals_conceeded_per_game"] = 0
    split_df["away_team_average_goals_scored_per_game"] = 0
    split_df["away_team_average_goals_conceeded_per_game"] = 0

    return split_df

def _create_split_averages_summary_template(split_df):
    '''create_summary_template 
    Function to create auxiliary template summary per decade

    Arguments:
        decade_df -- _description_

    Returns:
        _description_
    '''
    summary_df_template = pd.DataFrame(columns=
    [
        'team_name', 'league', 'total_goals_scored', 'total_goals_conceeded', 'games_played', 'goals_scored_per_game', 
        'goals_conceeded_per_game'
        ])
    team_group = split_df.groupby("Home_Team")
    home_teams = list(split_df["Home_Team"].unique())
    away_teams = list(split_df["Away_Team"].unique())
    missing_teams = list(set(away_teams) - set(home_teams))
    team_list = home_teams + missing_teams

    for team_name in team_list:
        league = team_group.first()["League"].unique()[0]
        team_stats_dict = {
            'team_name': team_name, 
            'league': league, 'total_goals_scored': 0, 
            'total_goals_conceeded': 0, 'games_played': 0, 'goals_scored_per_game':0, 
            'goals_conceeded_per_game':0
            }
        team_stats_dict_df = pd.DataFrame([team_stats_dict])
        summary_df_template = pd.concat([summary_df_template, team_stats_dict_df], ignore_index=True)


    print("Summary Template Complete")
    return summary_df_template

def populate_df_with_averages(split_df, summary_df_template):
    '''populate_df_with_averages 
    Function to compute and add average goals per game to combined dataframe.

    This is achieved by:
    1. Ordering split df by season and round.
    2. Splitting df into seasons
    3. For each season splitting into round
    4a. In each round split, if data exists, inputting into split df
    4b. This is put here so that averages for the round 1 before are put into round 2
    5. For each row in the round df goals per game is computed
    6. This is then added to a auxilary df where a running count of the averages are stored
    7. This is then updated into the split df for each match per round


    Arguments:
        split_df -- _description_

    Returns:
        _description_
    '''

    split_df = split_df.sort_values(['Season', 'Round'], ascending=[True, True])

    season_df_grouped = split_df.groupby("Season")

    for season in split_df["Season"].unique():
        season_df = season_df_grouped.get_group(season)
        round_df_grouped = season_df.groupby("Round")
        for round in season_df["Round"].unique():
            print(f"Starting round {round}, season {season}")

            round_df = round_df_grouped.get_group(round)
            # Set the values of the previous match
            for team in list(summary_df_template["team_name"]):

                populated_season_df = _add_home_team_feature_total_to_df("home_team_average_goals_scored_per_game","goals_scored_per_game", team, season_df, summary_df_template, round)
                populated_season_df = _add_home_team_feature_total_to_df("home_team_average_goals_conceeded_per_game","goals_conceeded_per_game", team, season_df, summary_df_template, round)
        
                populated_season_df = _add_away_team_feature_total_to_df("away_team_average_goals_scored_per_game","goals_scored_per_game", team, season_df, summary_df_template, round)
                populated_season_df = _add_away_team_feature_total_to_df("away_team_average_goals_conceeded_per_game","goals_conceeded_per_game", team, season_df, summary_df_template, round)
            

            for index, row in round_df.iterrows():

                home_team_name = row["Home_Team"]
                away_team_name = row["Away_Team"]
                league = row["League"]
                season=row["Season"]
                round=row["Round"]
                home_goals_scored = row["Home_Goals"]
                away_goals_scored = row["Away_Goals"]

                #Home Team
                summary_df_template = _goals_and_averages_computer(summary_df_template, home_team_name, home_goals_scored, 
                away_goals_scored, league)

                #Away Team
                summary_df_template = _goals_and_averages_computer(summary_df_template, away_team_name, away_goals_scored, 
                home_goals_scored, league)
        
        
        split_df = merge_season_df_with_combined_df(split_df, populated_season_df)

        print(f"Average Goals computed and added to df for season {season}")

    return split_df


def _goals_and_averages_computer(summary_df_template, team, goal_option_1, goal_option_2, league):
    '''_goals_and_averages_computer _summary_

    Arguments:
        summary_df_template -- _description_
        team -- _description_
        goal_option_1 -- _description_
        goal_option_2 -- _description_
        league -- _description_

    Returns:
        _description_
    '''

    summary_df_template.loc[((summary_df_template.team_name == team) & (
    summary_df_template.league == league)), "total_goals_scored"] += goal_option_1

    summary_df_template.loc[((summary_df_template.team_name == team) & (
    summary_df_template.league == league)), "total_goals_conceeded"] += goal_option_2

    summary_df_template.loc[((summary_df_template.team_name == team) & (
    summary_df_template.league == league)), "games_played"] += 1

    summary_df_template.loc[((summary_df_template.team_name == team) & (
    summary_df_template.league == league)), "goals_scored_per_game"] = summary_df_template.loc[((summary_df_template.team_name == team) & (
    summary_df_template.league == league)), "total_goals_scored"] / summary_df_template.loc[((summary_df_template.team_name == team) & (
    summary_df_template.league == league)), "games_played"]

    summary_df_template.loc[((summary_df_template.team_name == team) & (
    summary_df_template.league == league)), "goals_conceeded_per_game"] = summary_df_template.loc[((summary_df_template.team_name == team) & (
    summary_df_template.league == league)), "total_goals_conceeded"] / summary_df_template.loc[((summary_df_template.team_name == team) & (
    summary_df_template.league == league)), "games_played"]

    return summary_df_template


def _add_home_team_feature_total_to_df(feature,summary_feature, team, df, summary_df_template, round):
    '''_add_home_team_feature_total_to_df _summary_

    Arguments:
        feature -- _description_
        summary_feature -- _description_
        team -- _description_
        df -- _description_
        summary_df_template -- _description_
        round -- _description_

    Returns:
        _description_
    '''
    
    df.loc[(df.Home_Team == team) & ((
                df.Round == round)), feature] = summary_df_template.loc[(
                    summary_df_template.team_name == team), summary_feature].values[0]
    return df

def _add_away_team_feature_total_to_df(feature,summary_feature, team, df, summary_df_template, round):
    '''_add_away_team_feature_total_to_df _summary_

    Arguments:
        feature -- _description_
        summary_feature -- _description_
        team -- _description_
        df -- _description_
        summary_df_template -- _description_
        round -- _description_

    Returns:
        _description_
    '''
    
    df.loc[((df.Away_Team == team) & (
                df.Round == round)), feature] = summary_df_template.loc[((
                    summary_df_template.team_name == team)), summary_feature].values[0]
    return df

def merge_season_df_with_combined_df(split_df, populated_season_df):
    '''merge_season_df_with_combined_df _summary_

    Arguments:
        df -- _description_
        populated_season_df -- _description_

    Returns:
        _description_
    '''
    split_df.update(populated_season_df, overwrite=True)

    return split_df

def _prepare_dataset_for_ML_models(df):
    '''_prepare_dataset_for_ML_models 
    Function to prepare data for ML models. This includes:

    1. Removing columns containing non-numerical data
    2. Creating a goal difference column as the target
    3. Drop Home and Away Goals as that is the result.
    4. Drop fixtures that do not contain ELO scores

    Arguments:
        df -- _description_

    Returns:
        _description_
    '''


    df.drop(axis=1, columns=['Result','Round', 'Link', 'Home_Win', 'Away_Win', 'Date_New', 'Home_Team', 'Away_Team', 'Season', 'League', 'Referee', 'City', 'Country', 'Stadium', 'Pitch','Home_Yellow', 'Home_Red', 'Away_Yellow', 'Away_Red'], inplace=True)
    df["Goal_Difference"] = df["Home_Goals"] - df["Away_Goals"]
    df.drop(axis=1, columns=['Home_Goals', 'Away_Goals'], inplace=True)
    df.dropna(subset=["ELO_home"], inplace = True)
    
    return df





In [2]:
# SetUp Visual

pd.set_option('display.max_columns', None)

# Paths

# Modifications to the initial pipeline required to modify data and create required features

results_path = r'/Users/tom/Documents/Coding/AiCore/Projects/4. Football Match Outcome Predictor /Scraped Datasets/raw_data.csv'
results_df = pd.read_csv(results_path)
results_df.drop(results_df.index[results_df.ELO_Home == 'blank'], inplace=True)
to_predict_df = results_df.loc[results_df.Home_Goals == 'blank']
results_df.loc[(results_df.Season == 2023) & (results_df.Round == 8), "Home_Goals"] = 0
results_df.loc[(results_df.Season == 2023) & (results_df.Round == 8), "Away_Goals"] = 0
results_df.drop(results_df.index[results_df.Home_Goals == 'blank'], inplace=True)

## Caller Functions


cleaned_results_df = clean_all_dataframes(results_df)
cleaned_results_wf_df = add_features_to_results_df(cleaned_results_df)
# none of the ref info is used so could remove this or team info
missing_data_information_list = []
league="premier_league"

cleaned_results_with_features_df = create_new_feature_columns(cleaned_results_df)
for season in cleaned_results_with_features_df["Season"].unique():
    season_df = group_by_season_df(cleaned_results_with_features_df, season)
    summary_df_template = create_summary_template(season_df)
    summary_df_populated, populated_season_df = populate_season_stats(season_df, summary_df_template)
    summary_df_populated_with_streak, populated_with_streak_season_df, missing_data_information_list = populate_streak_season_stats(summary_df_populated, populated_season_df, season, league, missing_data_information_list)
    populated_results_df = merge_season_df_with_combined_df(cleaned_results_with_features_df, populated_with_streak_season_df)






Summary Template Complete
Summary DF Template Populated for Round 1
Summary DF Template Populated for Round 2
Summary DF Template Populated for Round 3
Summary DF Template Populated for Round 4
Summary DF Template Populated for Round 5
Summary DF Template Populated for Round 6
Summary DF Template Populated for Round 7
Summary DF Template Populated for Round 8
Summary DF Template Populated for Round 9
Summary DF Template Populated for Round 10
Summary DF Template Populated for Round 11
Summary DF Template Populated for Round 12
Summary DF Template Populated for Round 13
Summary DF Template Populated for Round 14
Summary DF Template Populated for Round 15
Summary DF Template Populated for Round 16
Summary DF Template Populated for Round 17
Summary DF Template Populated for Round 18
Summary DF Template Populated for Round 19
Summary DF Template Populated for Round 20
Summary DF Template Populated for Round 21
Summary DF Template Populated for Round 22
Summary DF Template Populated for Rou

In [6]:
#path = r'/Users/tom/Documents/Coding/AiCore/Projects/4. Football Match Outcome Predictor /Scraped Datasets/cleaned_populated_results_df.csv'
#populated_results_df.to_csv(path, index=False)

Milestone 3 - Feature Engineering

In [5]:
populated_results_df

,Home_Team,Away_Team,Season,Round,League,Home_Goals,Away_Goals,ELO_Home,ELO_Away,Home_Win,Away_Win,home_team_total_goals_scored_so_far,home_team_total_goals_conceeded_so_far,home_team_current_win_streak,home_team_current_loss_streak,home_team_total_points_so_far,home_team_current_goal_drought,home_team_total_wins_so_far,away_team_total_goals_scored_so_far,away_team_total_goals_conceeded_so_far,away_team_current_win_streak,away_team_current_loss_streak,away_team_total_points_so_far,away_team_current_goal_drought,away_team_total_wins_so_far
0,Fulham,Arsenal,2021.0,1.0,premier_league,0.0,3.0,69.0,90.0,False,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Crystal Palace,Southampton,2021.0,1.0,premier_league,1.0,0.0,74.0,79.0,True,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Liverpool,Leeds United,2021.0,1.0,premier_league,4.0,3.0,96.0,69.0,True,False,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,West Ham,Newcastle,2021.0,1.0,premier_league,0.0,2.0,76.0,74.0,False,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,West Bromwich Albion,Leicester,2021.0,1.0,premier_league,0.0,3.0,79.0,83.0,False,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
835,Tottenham Hotspur,Leicester,2023.0,8.0,premier_league,0.0,0.0,92.0,83.0,False,False,12.0,5.0,1.0,0.0,14.0,0.0,4.0,8.0,16.0,0.0,5.0,1.0,0.0,0.0
836,Brentford,Arsenal,2023.0,8.0,premier_league,0.0,0.0,62.0,95.0,False,False,15.0,9.0,1.0,0.0,9.0,0.0,2.0,14.0,7.0,0.0,1.0,15.0,0.0,5.0
837,Everton,West Ham,2023.0,8.0,premier_league,0.0,0.0,84.0,80.0,False,False,4.0,6.0,0.0,2.0,4.0,1.0,0.0,3.0,8.0,0.0,1.0,4.0,0.0,1.0
838,Man. Utd,Leeds United,2023.0,8.0,premier_league,0.0,0.0,93.0,62.0,False,False,8.0,8.0,4.0,0.0,12.0,0.0,4.0,10.0,10.0,0.0,2.0,8.0,0.0,2.0


In [4]:
populated_results_df.loc[(populated_results_df.Season == 2023) & ((populated_results_df.Home_Team == "Man. Utd") | (populated_results_df.Away_Team == "Man. Utd"))]

,Home_Team,Away_Team,Season,Round,League,Home_Goals,Away_Goals,ELO_Home,ELO_Away,Home_Win,Away_Win,home_team_total_goals_scored_so_far,home_team_total_goals_conceeded_so_far,home_team_current_win_streak,home_team_current_loss_streak,home_team_total_points_so_far,home_team_current_goal_drought,home_team_total_wins_so_far,away_team_total_goals_scored_so_far,away_team_total_goals_conceeded_so_far,away_team_current_win_streak,away_team_current_loss_streak,away_team_total_points_so_far,away_team_current_goal_drought,away_team_total_wins_so_far
768,Man. Utd,Brighton & Hove Albion,2023.0,1.0,premier_league,1.0,2.0,94.0,65.0,False,True,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
776,Brentford,Man. Utd,2023.0,2.0,premier_league,4.0,0.0,60.0,94.0,True,False,2.0,2.0,0.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0
789,Man. Utd,Liverpool,2023.0,3.0,premier_league,2.0,1.0,94.0,94.0,True,False,1.0,6.0,0.0,2.0,0.0,1.0,0.0,3.0,3.0,0.0,0.0,2.0,0.0,0.0
790,Southampton,Man. Utd,2023.0,4.0,premier_league,0.0,1.0,82.0,94.0,False,True,5.0,7.0,1.0,0.0,4.0,0.0,1.0,3.0,7.0,1.0,0.0,3.0,0.0,1.0
809,Leicester,Man. Utd,2023.0,5.0,premier_league,0.0,1.0,83.0,94.0,False,True,6.0,10.0,0.0,3.0,1.0,0.0,0.0,4.0,7.0,2.0,0.0,6.0,0.0,2.0
819,Man. Utd,Arsenal,2023.0,6.0,premier_league,3.0,1.0,94.0,95.0,True,False,5.0,7.0,3.0,0.0,9.0,0.0,3.0,13.0,4.0,5.0,0.0,15.0,0.0,5.0
838,Man. Utd,Leeds United,2023.0,8.0,premier_league,0.0,0.0,93.0,62.0,False,False,8.0,8.0,4.0,0.0,12.0,0.0,4.0,10.0,10.0,0.0,2.0,8.0,0.0,2.0
